# <p style="background-color:#186F65;font-family:newtimeroman;color:#F4A460;font-size:150%;text-align:center;border-radius:40px 40px;">UM - Game | Starter </p>

In [ ]:
import numpy as np
import polars as pl
import pandas as pd
from pathlib import Path

import gc
import os
import sys

from tqdm import tqdm
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

import kaggle_evaluation.mcts_inference_server
import mcts_inference_server
pd.options.display.max_columns = None

import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import *
from sklearn.metrics import mean_squared_error

# <p style="background-color:#186F65;font-family:newtimeroman;color:#F4A460;font-size:150%;text-align:center;border-radius:40px 40px;">Load Data and Some Basic Preprocessing </p>

In [ ]:
%%time

train = pl.read_csv('/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv')
test = pl.read_csv('/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv')

group_col = train['GameRulesetName']

In [ ]:
%%time

def remove_useless(df: pl.DataFrame) -> pl.DataFrame:
    Dropped = ['Id', 'Properties', 'Format', 'Time', 'Discrete', 'Realtime', 'Turns', 'Alternating',
        'Simultaneous', 'HiddenInformation', 'Match', 'AsymmetricRules', 'AsymmetricPlayRules',
        'AsymmetricEndRules', 'AsymmetricSetup', 'Players', 'NumPlayers', 'Simulation', 'Solitaire',
        'TwoPlayer', 'Multiplayer', 'Coalition', 'Puzzle', 'DeductionPuzzle', 'PlanningPuzzle', 'Equipment', 
        'Container', 'Board', 'PrismShape', 'ParallelogramShape', 'RectanglePyramidalShape', 'TargetShape', 
        'BrickTiling', 'CelticTiling', 'QuadHexTiling', 'Hints', 'PlayableSites', 'Component', 'DiceD3',
        'BiasedDice', 'Card', 'Domino', 'Rules', 'SituationalTurnKo', 'SituationalSuperko', 'InitialAmount',
        'InitialPot', 'Play', 'BetDecision', 'BetDecisionFrequency', 'VoteDecisionFrequency',
        'ChooseTrumpSuitDecision', 'ChooseTrumpSuitDecisionFrequency', 'LeapDecisionToFriend', 
        'LeapDecisionToFriendFrequency', 'HopDecisionEnemyToFriend', 'HopDecisionEnemyToFriendFrequency',
        'HopDecisionFriendToFriend', 'FromToDecisionWithinBoard', 'FromToDecisionBetweenContainers', 'BetEffect',
        'BetEffectFrequency', 'VoteEffectFrequency', 'SwapPlayersEffectFrequency', 'TakeControl',
        'TakeControlFrequency', 'PassEffectFrequency', 'SetCost', 'SetCostFrequency', 'SetPhase', 
        'SetPhaseFrequency', 'SetTrumpSuit', 'SetTrumpSuitFrequency', 'StepEffectFrequency', 
        'SlideEffectFrequency', 'LeapEffectFrequency', 'HopEffectFrequency', 'FromToEffectFrequency',
        'SwapPiecesEffect', 'SwapPiecesEffectFrequency', 'ShootEffect', 'ShootEffectFrequency', 'MaxCapture',
        'OffDiagonalDirection', 'Information', 'HidePieceType', 'HidePieceOwner', 'HidePieceCount',
        'HidePieceRotation', 'HidePieceValue', 'HidePieceState', 'InvisiblePiece', 'End', 'LineDrawFrequency',
        'ConnectionDraw', 'ConnectionDrawFrequency', 'GroupLossFrequency', 'GroupDrawFrequency', 
        'LoopLossFrequency', 'LoopDraw', 'LoopDrawFrequency', 'PatternLoss', 'PatternLossFrequency', 
        'PatternDraw', 'PatternDrawFrequency', 'PathExtentEndFrequency', 'PathExtentWinFrequency',
        'PathExtentLossFrequency', 'PathExtentDraw', 'PathExtentDrawFrequency', 'TerritoryLoss',
        'TerritoryLossFrequency', 'TerritoryDraw', 'TerritoryDrawFrequency', 'CheckmateLoss', 
        'CheckmateLossFrequency', 'CheckmateDraw', 'CheckmateDrawFrequency', 'NoTargetPieceLoss', 
        'NoTargetPieceLossFrequency', 'NoTargetPieceDraw', 'NoTargetPieceDrawFrequency', 'NoOwnPiecesDraw',
        'NoOwnPiecesDrawFrequency', 'FillLoss', 'FillLossFrequency', 'FillDraw', 'FillDrawFrequency', 
        'ScoringDrawFrequency', 'NoProgressWin', 'NoProgressWinFrequency', 'NoProgressLoss', 
        'NoProgressLossFrequency', 'SolvedEnd', 'Behaviour', 'StateRepetition', 'PositionalRepetition',
        'SituationalRepetition', 'Duration', 'Complexity', 'BoardCoverage', 'GameOutcome', 'StateEvaluation',
        'Clarity', 'Narrowness', 'Variance', 'Decisiveness', 'DecisivenessMoves', 'DecisivenessThreshold', 
        'LeadChange', 'Stability', 'Drama', 'DramaAverage', 'DramaMedian', 'DramaMaximum', 'DramaMinimum', 
        'DramaVariance', 'DramaChangeAverage', 'DramaChangeSign', 'DramaChangeLineBestFit', 'DramaChangeNumTimes', 
        'DramaMaxIncrease', 'DramaMaxDecrease', 'MoveEvaluation', 'MoveEvaluationAverage', 'MoveEvaluationMedian',
        'MoveEvaluationMaximum', 'MoveEvaluationMinimum', 'MoveEvaluationVariance', 'MoveEvaluationChangeAverage',
        'MoveEvaluationChangeSign', 'MoveEvaluationChangeLineBestFit', 'MoveEvaluationChangeNumTimes',
        'MoveEvaluationMaxIncrease', 'MoveEvaluationMaxDecrease', 'StateEvaluationDifference',
        'StateEvaluationDifferenceAverage', 'StateEvaluationDifferenceMedian', 'StateEvaluationDifferenceMaximum', 
        'StateEvaluationDifferenceMinimum', 'StateEvaluationDifferenceVariance', 'StateEvaluationDifferenceChangeAverage',
        'StateEvaluationDifferenceChangeSign', 'StateEvaluationDifferenceChangeLineBestFit', 
        'StateEvaluationDifferenceChangeNumTimes', 'StateEvaluationDifferenceMaxIncrease', 
        'StateEvaluationDifferenceMaxDecrease', 'BoardSitesOccupied', 'BoardSitesOccupiedMinimum', 
        'BranchingFactor', 'BranchingFactorMinimum', 'DecisionFactor', 'DecisionFactorMinimum', 'MoveDistance',
        'MoveDistanceMinimum', 'PieceNumber', 'PieceNumberMinimum', 'ScoreDifference', 'ScoreDifferenceMinimum', 
        'ScoreDifferenceChangeNumTimes', 'Roots', 'Cosine', 'Sine', 'Tangent', 'Exponential', 'Logarithm', 
        'ExclusiveDisjunction', 'Float', 'HandComponent', 'SetHidden', 'SetInvisible', 'SetHiddenCount', 'SetHiddenRotation',
        'SetHiddenState', 'SetHiddenValue', 'SetHiddenWhat', 'SetHiddenWho','GameRulesetName', 'EnglishRules', 'LudRules']
    
    Dropped = [col for col in Dropped if col in df.columns]
    
    df = df.drop(Dropped)
    
    return df

more_drops = ['num_wins_agent1', 'num_draws_agent1', 'num_losses_agent1']
more_drops = [col for col in more_drops if col in train.columns]

train = train.drop(more_drops)
train = remove_useless(train)

In [ ]:
%%time

agent_cols = ['agent1', 'agent2']

def process_agent_cols(df, agent_cols):

    for col in agent_cols:
        split_cols = df[col].str.split(by="-").to_list()
        max_splits = max(len(lst) for lst in split_cols)
        
        for idx in range(max_splits):
            new_col_name = f"{col}_{idx}"
            df = df.with_columns(
                pl.DataFrame({new_col_name: [lst[idx] if idx < len(lst) else None for lst in split_cols]})
            )

        df = df.drop(col)
        
    df = df.with_columns(
        [pl.col(col).cast(pl.Categorical) for col in df.columns if any(col.startswith(agent) for agent in agent_cols)]
    )
    
    df = df.with_columns(
        [pl.col(col).cast(pl.Float32) for col in df.columns if not any(col.startswith(agent) for agent in agent_cols)]
    )
    
    return df.to_pandas()

train = process_agent_cols(train, agent_cols)

In [ ]:
%%time

def encode_Train(df):
    df['agent1_0'] = df['agent1_0'].map({'MCTS': 0}).astype(int)
    df['agent1_1'] = df['agent1_1'].map({'UCB1Tuned': 0, 'UCB1GRAVE': 1, 'UCB1': 3, 'ProgressiveHistory': 4}).astype(int)
    df['agent1_2'] = df['agent1_2'].map({'0.1': 0, '0.6': 1, '1.41421356237': 2}).astype(int)
    df['agent1_3'] = df['agent1_3'].map({'Random200': 0, 'MAST': 1, 'NST': 3}).astype(int)
    df['agent1_4'] = df['agent1_4'].map({'false': 0, 'true': 1}).astype(int)
    
    df['agent2_0'] = df['agent2_0'].map({'MCTS': 0}).astype(int)
    df['agent2_1'] = df['agent2_1'].map({'UCB1Tuned': 0, 'UCB1GRAVE': 1, 'UCB1': 3, 'ProgressiveHistory': 4}).astype(int)
    df['agent2_2'] = df['agent2_2'].map({'0.1': 0, '0.6': 1, '1.41421356237': 2}).astype(int)
    df['agent2_3'] = df['agent2_3'].map({'Random200': 0, 'MAST': 1, 'NST': 3}).astype(int)
    df['agent2_4'] = df['agent2_4'].map({'false': 0, 'true': 1}).astype(int)
    
    return df 

def encode_Test(df):
    df['agent1_0'] = df['agent1_0'].map({'MCTS': 0}).astype(int)
    df['agent1_1'] = df['agent1_1'].map({'UCB1Tuned': 0, 'UCB1': 3, 'ProgressiveHistory': 4}).astype(int)
    df['agent1_2'] = df['agent1_2'].map({'0.1': 0, '0.6': 1}).astype(int)
    df['agent1_3'] = df['agent1_3'].map({'Random200': 0, 'MAST': 1, 'NST': 3}).astype(int)
    df['agent1_4'] = df['agent1_4'].map({'false': 0, 'true': 1}).astype(int)
    
    df['agent2_0'] = df['agent2_0'].map({'MCTS': 0}).astype(int)
    df['agent2_1'] = df['agent2_1'].map({'UCB1': 3, 'ProgressiveHistory': 4}).astype(int)
    df['agent2_2'] = df['agent2_2'].map({"0.1": 0, "0.6": 1}).astype(int)
    df['agent2_3'] = df['agent2_3'].map({'MAST': 1, 'NST': 3}).astype(int)
    df['agent2_4'] = df['agent2_4'].map({'false': 0, 'true': 1}).astype(int)
    
    return df 

train = encode_Train(train)

In [ ]:
%%time

train.head()

# <p style="background-color:#186F65;font-family:newtimeroman;color:#F4A460;font-size:150%;text-align:center;border-radius:40px 40px;">Modeling | LGBM </p>

In [ ]:
%%time

X = train.drop(['utility_agent1'], axis=1)
y = train['utility_agent1']
groups = group_col

cat_cols = [col for col in X.columns if X[col].dtype == 'category']

SEED = 42
n_splits = 5

def TrainML(model):
    
    group_kfold = GroupKFold(n_splits=n_splits)
    
    train_rmse_scores = []
    test_rmse_scores = []
    trained_models = []
    
    for fold, (train_idx, test_idx) in enumerate(tqdm(group_kfold.split(X, y, groups=groups), 
                                                      desc="Training Folds", total=n_splits)):
        
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        model.fit(X_train, y_train)
        trained_models.append(model)
        
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        
        train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
        
        train_rmse_scores.append(train_rmse)
        test_rmse_scores.append(test_rmse)
        
        print(f"Fold {fold+1} - Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}")
        clear_output(wait=True)
    
    print("\n--- Final Mean Scores ---")
    print(f"Mean Train RMSE: {np.mean(train_rmse_scores):.4f}")
    print(f"Mean Test RMSE: {np.mean(test_rmse_scores):.4f}")
    
    results_df = pd.DataFrame({
        'Fold': np.arange(1, n_splits+1),
        'Train RMSE': train_rmse_scores,
        'Test RMSE': test_rmse_scores
    })
    
    print("\n=== KFold RMSE Results ===")
    print(results_df)
    
    return trained_models, model

In [ ]:
%%time

cparams = {'iterations': 880, 'learning_rate': 0.09106680282445292, 'depth': 9, 'l2_leaf_reg': 2.3092232729544727e-07, 
             'min_data_in_leaf': 40, 'random_strength': 0.2104300091500597, 'bagging_temperature': 0.0016575151109741422,
             'grow_policy': 'SymmetricTree'} #0.4312

c_p = {'iterations': 1844, 'learning_rate': 0.058375587542823536, 'depth': 10, 'l2_leaf_reg': 0.000642543780540334,
       'min_data_in_leaf': 24, 'random_strength': 1.0783250897617306e-07, 'bagging_temperature': 0.042308185479400635,
       'grow_policy': 'SymmetricTree'} # 0.4244

L_p = {'learning_rate': 0.032278310677039956, 'num_leaves': 208, 'n_estimators': 1659, 
       'min_child_samples': 42, 'subsample': 0.7998561811258824, 'colsample_bytree': 0.915664341770982, 
       'reg_alpha': 0.01453093119952141, 'reg_lambda': 0.00014726830932600127, 'scale_pos_weight': 1.455335307825233,
       'min_split_gain': 1.289977216814962e-06, 'min_child_weight': 0.0012167996364537176} #0.4301

Light = lgb.LGBMRegressor(**L_p, random_state=SEED, verbose=-1, device='gpu')
Cat = CatBoostRegressor(**c_p, random_state=SEED, verbose=0,task_type='GPU')
Cat1 = CatBoostRegressor(**cparams, random_state=SEED, verbose=0,task_type='GPU')

estimator = VotingRegressor(
    estimators=[
        ('CAT', Cat),
        ('Light', Light),
        ('CAT_1', Cat1),
    ],
    weights=[0.7,0.5,0.3]
)

# <p style="background-color:#186F65;font-family:newtimeroman;color:#F4A460;font-size:150%;text-align:center;border-radius:40px 40px;">Submission</p>

**Perform All Preprocessing in a Single Function. During Submission, We Need That Function to Clean Text Data. If You Perform Preprocessing Separately, It Will Not Be Helpful. You Need to Ensure That the Preprocessing Function is Incorporated Into the Prediction Function. This is a Small Guide to Avoid Errors During Submission.**

In [ ]:
%%time

Light_Models = None
counter = 0

def train_model():
    global Light_Models
    
    if Light_Models is None:
        Light_Models, Light_m = TrainML(estimator)

def infer_lgb(data, models):
    return np.mean([model.predict(data) for model in models], axis=0)

def predict(test, submission):
    global Light_Models, agent_cols, counter
    
    train_model() if counter == 0 else None
    
    counter += 1
    
    test = remove_useless(test)
    test = process_agent_cols(test, agent_cols)
    test = encode_Train(test)    
    
    return submission.with_columns(pl.Series('utility_agent1', infer_lgb(test, Light_Models)))

inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
        )
    )